<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IFC-Modellierung mit IfcOpenShell - Vollständiges Tutorial

## Lernziele
In diesem Tutorial lernen Sie:
- Ein vollständiges IFC-Modell von Grund auf zu erstellen
- Wände mit korrekter Geometrie und Eigenschaften zu modellieren
- Fenster mit Öffnungen und Materialien hinzuzufügen
- Property Sets (Pset) für erweiterte Gebäudeinformationen zu verwenden
- Beziehungen zwischen Bauteilen korrekt zu definieren
- IFC-Standards und Best Practices anzuwenden

### Einrichtung und Installation
Bevor wir mit der Bearbeitung von IFC-Modellen beginnen, müssen wir sicherstellen, dass `ifcopenshell` installiert ist. Diese Bibliothek ermöglicht es uns, IFC-Dateien effektiv zu bearbeiten und zu analysieren.


In [ ]:
%pip install ifcopenshell

: 

## Schritt 1: Vollständiges IFC-Modell mit Wand und Fenster erstellen

### Was passiert in diesem Schritt?
Wir erstellen ein komplettes IFC-Modell von Grund auf, das alle notwendigen Elemente enthält:

1. **Projektstruktur**: Projekt → Standort → Gebäude → Geschoss
2. **Geometrische Kontexte**: 3D-Modellierungsumgebung definieren
3. **Einheiten**: Metrische Einheiten (Meter, Quadratmeter, Kubikmeter)
4. **Wand**: Mit korrekter Geometrie und Platzierung
5. **Fenster**: Mit Fenstertyp, Eigenschaften und Öffnung in der Wand
6. **Property Sets**: Erweiterte Eigenschaften für Wände und Fenster
7. **Beziehungen**: Korrekte Verknüpfungen zwischen allen Elementen

### Warum ist das wichtig?
- **IFC-Standards**: Vollständige Kompatibilität mit IFC4-Schema
- **BIM-Software**: Korrekte Darstellung in allen gängigen BIM-Viewern
- **Datenqualität**: Strukturierte und standardisierte Gebäudeinformationen
- **Interoperabilität**: Austausch zwischen verschiedenen Software-Plattformen

In [ ]:
import ifcopenshell
import datetime
import time

def create_complete_ifc_model():
    """Erstellt ein vollständiges IFC-Modell mit Wand, Fenster und Property Sets"""
    
    print("🚀 Erstelle IFC-Modell mit Wand und Fenster...")
    
    # 1. GRUNDSTRUKTUR: Leeres IFC-Modell erstellen
    model = ifcopenshell.file()
    print("✅ Leeres IFC-Modell erstellt")
    
    # 2. OWNER HISTORY: Für Nachverfolgung von Änderungen (IFC-Standard)
    org = model.createIfcOrganization(None, "BFH Architektur", None, None, None)
    person = model.createIfcPerson(None, "Student", "BIM", None, None, None, None, None)
    person_org = model.createIfcPersonAndOrganization(person, org, None)
    app = model.createIfcApplication(org, "0.8", "IfcOpenShell", "ifcopenshell")
    owner_history = model.createIfcOwnerHistory(
        person_org, app, None, "NOCHANGE", None, None, None, int(time.time())
    )
    
    # 3. PROJEKT: Hauptcontainer für alle Gebäudedaten
    project = model.createIfcProject(
        ifcopenshell.guid.new(), 
        owner_history, 
        "BFH Beispielprojekt", 
        "Demonstrationsprojekt für IFC-Modellierung",
        None, 
        "BFH Beispielprojekt - Vollversion", 
        "Entwurf", 
        None, 
        None
    )
    print("✅ Projekt erstellt")
    
    # 4. EINHEITEN: Metrisches System definieren
    length_unit = model.createIfcSIUnit(None, "LENGTHUNIT", None, "METRE")
    area_unit = model.createIfcSIUnit(None, "AREAUNIT", None, "SQUARE_METRE")
    volume_unit = model.createIfcSIUnit(None, "VOLUMEUNIT", None, "CUBIC_METRE")
    unit_assignment = model.createIfcUnitAssignment([length_unit, area_unit, volume_unit])
    print("✅ Metrische Einheiten definiert")
    
    # 5. GEOMETRISCHE KONTEXTE: 3D-Modellierungsumgebung
    world_origin = model.createIfcCartesianPoint((0.0, 0.0, 0.0))
    world_z = model.createIfcDirection((0.0, 0.0, 1.0))  # Z-Achse nach oben
    world_x = model.createIfcDirection((1.0, 0.0, 0.0))  # X-Achse nach rechts
    world_placement = model.createIfcAxis2Placement3D(world_origin, world_z, world_x)
    
    # Hauptkontext für 3D-Modellierung
    geom_context = model.createIfcGeometricRepresentationContext(
        None, "Model", 3, 1e-05, world_placement, world_z
    )
    # Unterkontext für Baukörper
    body_context = model.createIfcGeometricRepresentationSubContext(
        "Body", "Model", None, None, None, None, geom_context, None, "MODEL_VIEW", None
    )
    print("✅ Geometrische Kontexte erstellt")
    
    # 6. PROJEKT MIT EINHEITEN UND KONTEXTEN VERKNÜPFEN
    project.OwnerHistory = owner_history
    project.RepresentationContexts = [geom_context]
    project.UnitsInContext = unit_assignment
    
    # 7. RÄUMLICHE HIERARCHIE: Projekt → Standort → Gebäude → Geschoss
    site = model.createIfcSite(
        ifcopenshell.guid.new(), owner_history, "BFH Campus", 
        "Hauptstandort der Berner Fachhochschule", None, world_placement, 
        None, "BFH Campus Biel", "ELEMENT", 
        (47, 8, 15, 123456), (7, 15, 30, 654321), 430.0, "BFH-001", None
    )
    
    building = model.createIfcBuilding(
        ifcopenshell.guid.new(), owner_history, "Hauptgebäude", 
        "Architektur und Bauingenieurwesen", None, world_placement, 
        None, "Hauptgebäude BFH-AHB", "ELEMENT", 430.0, 430.0, None
    )
    
    storey = model.createIfcBuildingStorey(
        ifcopenshell.guid.new(), owner_history, "Erdgeschoss", 
        "Erdgeschoss mit Unterrichtsräumen", None, world_placement, 
        None, "Erdgeschoss (EG)", "ELEMENT", 0.0
    )
    
    # Hierarchie-Beziehungen erstellen
    model.createIfcRelAggregates(ifcopenshell.guid.new(), owner_history, "Projekt enthält Standort", None, project, [site])
    model.createIfcRelAggregates(ifcopenshell.guid.new(), owner_history, "Standort enthält Gebäude", None, site, [building])
    model.createIfcRelAggregates(ifcopenshell.guid.new(), owner_history, "Gebäude enthält Geschoss", None, building, [storey])
    print("✅ Räumliche Hierarchie erstellt (Projekt → Standort → Gebäude → Geschoss)")
    
    return model, owner_history, body_context, storey, world_placement

# Modell erstellen
model, owner_history, body_context, storey, world_placement = create_complete_ifc_model()

## Schritt 2: Wand mit Geometrie erstellen

### Was passiert hier?
1. **Wandelement**: Erstellen einer `IfcWall` mit korrektem Typ
2. **Geometrie**: Rechteckiges Profil wird zu einem 3D-Körper extrudiert
3. **Abmessungen**: 5m lang × 0.2m dick × 3m hoch
4. **Platzierung**: Positionierung im 3D-Raum
5. **Zuordnung**: Wand wird dem Geschoss zugeordnet
6. **Wandtyp**: Definition eines Wandtyps für bessere Klassifizierung


In [ ]:
def create_wall_with_geometry(model, owner_history, body_context, storey, world_placement):
    """Erstellt eine Wand mit korrekter Geometrie und Eigenschaften"""
    
    print("🧱 Erstelle Wand mit Geometrie...")
    
    # 1. WAND ERSTELLEN: IfcWall mit Grundeigenschaften
    wall = model.createIfcWall(
        ifcopenshell.guid.new(),      # Eindeutige ID
        owner_history,                # Änderungshistorie
        "Außenwand Nord",             # Name
        "Hauptaußenwand des Gebäudes", # Beschreibung
        None,                         # Objekttyp (optional)
        world_placement,              # Platzierung
        None,                         # Geometrie (wird später zugewiesen)
        None,                         # Tag (optional)
        "STANDARD"                    # Vordefinierter Typ
    )
    
    # 2. WANDGEOMETRIE: Rechteckiges Profil für Extrusion
    # Profil definiert den Querschnitt der Wand (Länge × Dicke)
    wall_profile = model.createIfcRectangleProfileDef(
        "AREA",           # Profiltyp (Fläche)
        "Wandprofil",     # Name des Profils
        world_placement,  # 2D-Achsensystem
        5.0,              # Länge: 5 Meter
        0.2               # Dicke: 20 Zentimeter
    )
    
    # 3. EXTRUSION: Profil wird in Z-Richtung (nach oben) extrudiert
    extrude_dir = model.createIfcDirection((0.0, 0.0, 1.0))  # Z-Richtung
    wall_solid = model.createIfcExtrudedAreaSolid(
        wall_profile,     # Zu extrudierendes Profil
        world_placement,  # Basis-Platzierung
        extrude_dir,      # Extrusionsrichtung
        3.0               # Höhe: 3 Meter
    )
    
    # 4. GEOMETRISCHE REPRÄSENTATION: 3D-Form definieren
    wall_shape = model.createIfcShapeRepresentation(
        body_context,     # Geometrischer Kontext
        "Body",           # Repräsentationstyp
        "SweptSolid",     # Geometriemethode (Extrusion)
        [wall_solid]      # Geometrische Elemente
    )
    
    # 5. PRODUKTDEFINITION: Geometrie mit Wand verknüpfen
    wall_definition = model.createIfcProductDefinitionShape(
        None,             # Name (optional)
        None,             # Beschreibung (optional)
        [wall_shape]      # Geometrische Repräsentationen
    )
    
    # Geometrie der Wand zuweisen
    wall.Representation = wall_definition
    print("✅ Wandgeometrie erstellt (5m × 0.2m × 3m)")
    
    # 6. WANDTYP: Klassifizierung für bessere Organisation
    wall_type = model.createIfcWallType(
        ifcopenshell.guid.new(),  # Eindeutige ID
        owner_history,            # Änderungshistorie
        "Außenwand Standard",     # Name des Typs
        "Standard-Außenwand 20cm", # Beschreibung
        None, None, None, None,   # Optionale Attribute
        "Außenwand Standard",     # Elementtyp
        "STANDARD"                # Vordefinierter Typ
    )
    
    # 7. TYP-BEZIEHUNG: Wand mit Wandtyp verknüpfen
    model.createIfcRelDefinesByType(
        ifcopenshell.guid.new(),  # Eindeutige ID
        owner_history,            # Änderungshistorie
        "Wandtyp-Zuweisung",     # Name der Beziehung
        None,                     # Beschreibung
        [wall],                   # Betroffene Objekte (Wandinstanzen)
        wall_type                 # Definierender Typ
    )
    print("✅ Wandtyp erstellt und zugewiesen")
    
    # 8. RÄUMLICHE ZUORDNUNG: Wand dem Geschoss zuordnen
    model.createIfcRelContainedInSpatialStructure(
        ifcopenshell.guid.new(),  # Eindeutige ID
        owner_history,            # Änderungshistorie
        "Wand im Geschoss",       # Name der Beziehung
        None,                     # Beschreibung
        [wall],                   # Enthaltene Elemente
        storey                    # Räumliche Struktur (Geschoss)
    )
    print("✅ Wand dem Geschoss zugeordnet")
    
    return wall

# Wand erstellen
wall = create_wall_with_geometry(model, owner_history, body_context, storey, world_placement)


## Schritt 3: Fenster mit Öffnung erstellen

### Was passiert hier?
1. **Fensterparameter**: Definition der Abmessungen und Position
2. **Fenstertyp**: Erstellung eines `IfcWindowType` mit Eigenschaften
3. **Fenstergeometrie**: 3D-Körper für das Fenster
4. **Platzierung**: Positionierung relativ zur Wand
5. **Wandöffnung**: `IfcOpeningElement` für den Durchbruch
6. **Beziehungen**: Verknüpfung zwischen Wand, Öffnung und Fenster
7. **Styling**: Farbe und Transparenz für bessere Visualisierung


In [ ]:
def create_window_with_opening(model, owner_history, body_context, storey, wall, world_placement):
    """Erstellt ein Fenster mit Wandöffnung und allen notwendigen Eigenschaften"""
    
    print("🪟 Erstelle Fenster mit Wandöffnung...")
    
    # 1. FENSTERPARAMETER: Abmessungen und Position definieren
    # Alle Werte in Millimetern, werden später in Meter konvertiert
    window_params = {
        "name": "Hauptfenster",
        "width": 1200,      # Breite: 1.2m
        "depth": 150,       # Tiefe: 15cm (dünner als Wand)
        "height": 1400,     # Höhe: 1.4m
        "offset_x": 1900,   # Position von links: 1.9m
        "offset_y": 25,     # Tiefenversatz: 2.5cm
        "offset_z": 800     # Höhe über Boden: 80cm
    }
    
    # Hilfsfunktion: Millimeter zu Meter
    to_m = lambda v_mm: float(v_mm) / 1000.0
    
    # 2. FENSTERPLATZIERUNG: Position relativ zur Wand
    window_point = model.createIfcCartesianPoint((
        to_m(window_params["offset_x"]),  # X: 1.9m von Wandanfang
        to_m(window_params["offset_y"]),  # Y: 2.5cm in die Wand
        to_m(window_params["offset_z"])   # Z: 80cm über dem Boden
    ))
    window_placement = model.createIfcLocalPlacement(
        world_placement,  # Bezugssystem (Welt-Koordinaten)
        model.createIfcAxis2Placement3D(window_point)
    )
    print("✅ Fensterposition berechnet")
    
    # 3. FENSTERGEOMETRIE: 3D-Körper erstellen
    # 2D-Profil für die Extrusion
    profile_origin = model.createIfcCartesianPoint((0.0, 0.0))
    profile_placement = model.createIfcAxis2Placement2D(profile_origin)
    
    window_profile = model.createIfcRectangleProfileDef(
        "AREA",                           # Profiltyp
        "Fensterprofil",                 # Name
        profile_placement,                # 2D-Platzierung
        to_m(window_params["width"]),     # Breite in Metern
        to_m(window_params["depth"])      # Tiefe in Metern
    )
    
    # Extrusion in Z-Richtung (Höhe)
    extrude_dir = model.createIfcDirection((0.0, 0.0, 1.0))
    window_solid = model.createIfcExtrudedAreaSolid(
        window_profile,                   # Zu extrudierendes Profil
        profile_placement,                # Basis-Platzierung
        extrude_dir,                      # Extrusionsrichtung
        to_m(window_params["height"])     # Höhe in Metern
    )
    
    # Geometrische Repräsentation
    window_shape = model.createIfcShapeRepresentation(
        body_context, "Body", "SweptSolid", [window_solid]
    )
    window_definition = model.createIfcProductDefinitionShape(None, None, [window_shape])
    print("✅ Fenstergeometrie erstellt")
    
    # 4. FENSTERTYP: Definition mit Eigenschaften
    window_type = model.createIfcWindowType(
        ifcopenshell.guid.new(),     # Eindeutige ID
        owner_history,               # Änderungshistorie
        "Standard Fenster",          # Name
        "Einfachverglasung mit Rahmen", # Beschreibung
        None,                        # Anwendungsbereich
        None,                        # Property Sets (später)
        None,                        # Repräsentationsmaps
        None,                        # Tag
        "Standard Fenster",          # Elementtyp
        "WINDOW",                    # Vordefinierter Typ
        "SINGLE_PANEL",              # Aufteilungstyp (einfach)
        True,                        # Parameter haben Vorrang
        None                         # Benutzerdefinierter Typ
    )
    
    # 5. FENSTER-EIGENSCHAFTEN: Rahmen und Panel definieren
    # Rahmeneigenschaften (Lining Properties)
    lining_props = model.createIfcWindowLiningProperties(
        ifcopenshell.guid.new(), owner_history, "Fensterrahmen", None,
        to_m(60),   # Rahmen-Tiefe: 6cm
        to_m(30),   # Rahmen-Dicke: 3cm  
        to_m(15),   # Sprosse-Dicke: 1.5cm
        to_m(15),   # Pfosten-Dicke: 1.5cm
        None, None, None, None, None,  # Optionale Offsets
        to_m(80),   # Rahmen-Versatz: 8cm
        to_m(15),   # Panel-Versatz X: 1.5cm
        to_m(15)    # Panel-Versatz Y: 1.5cm
    )
    
    # Panel-Eigenschaften (Operational Properties)
    panel_props = model.createIfcWindowPanelProperties(
        ifcopenshell.guid.new(), owner_history, "Fensterpanel", None,
        "SIDEHUNGRIGHTHAND",  # Öffnungsart: seitlich rechts
        "RIGHT",              # Panel-Position
        to_m(window_params["width"]),   # Frame-Tiefe
        to_m(window_params["height"]),  # Frame-Dicke  
        None                  # Form-Aspekt
    )
    
    # Eigenschaften dem Fenstertyp zuordnen
    window_type.HasPropertySets = [lining_props, panel_props]
    print("✅ Fenstertyp mit Eigenschaften erstellt")
    
    # 6. FENSTER-INSTANZ: Konkretes Fenster erstellen
    window = model.createIfcWindow(
        ifcopenshell.guid.new(),          # Eindeutige ID
        owner_history,                    # Änderungshistorie
        window_params["name"],            # Name
        "Hauptfenster der Nordfassade",   # Beschreibung
        None,                             # Objekttyp
        window_placement,                 # Platzierung
        window_definition,                # Geometrie
        None,                             # Tag
        to_m(window_params["height"]),    # Gesamthöhe
        to_m(window_params["width"]),     # Gesamtbreite
        "WINDOW",                         # Vordefinierter Typ
        "SINGLE_PANEL",                   # Aufteilungstyp
        None                              # Benutzerdefiniert
    )
    
    # 7. TYP-BEZIEHUNG: Fenster mit Fenstertyp verknüpfen
    model.createIfcRelDefinesByType(
        ifcopenshell.guid.new(), owner_history, 
        "Fenstertyp-Zuweisung", None, [window], window_type
    )
    
    # 8. RÄUMLICHE ZUORDNUNG: Fenster dem Geschoss zuordnen
    model.createIfcRelContainedInSpatialStructure(
        ifcopenshell.guid.new(), owner_history,
        "Fenster im Geschoss", None, [window], storey
    )
    print("✅ Fenster erstellt und zugeordnet")
    
    return window, window_params, to_m

# Fenster erstellen
window, window_params, to_m = create_window_with_opening(model, owner_history, body_context, storey, wall, world_placement)


## Schritt 4: Wandöffnung und Styling

### Was passiert hier?
1. **Wandöffnung**: `IfcOpeningElement` für den Durchbruch in der Wand
2. **Void-Beziehung**: Verknüpfung zwischen Wand und Öffnung
3. **Fill-Beziehung**: Verknüpfung zwischen Öffnung und Fenster
4. **Farbgebung**: Blaue, semi-transparente Darstellung des Fensters
5. **IFC-Konformität**: Korrekte Beziehungen nach IFC4-Standard


In [ ]:
def create_wall_opening_and_styling(model, owner_history, body_context, wall, window, window_params, to_m, world_placement):
    """Erstellt Wandöffnung und fügt Styling hinzu"""
    
    print("🔲 Erstelle Wandöffnung und Styling...")
    
    # 1. ÖFFNUNGSPLATZIERUNG: Gleiche Position wie Fenster
    opening_point = model.createIfcCartesianPoint((
        to_m(window_params["offset_x"]),
        to_m(window_params["offset_y"]), 
        to_m(window_params["offset_z"])
    ))
    opening_placement = model.createIfcLocalPlacement(
        world_placement,
        model.createIfcAxis2Placement3D(opening_point)
    )
    
    # 2. ÖFFNUNGSGEOMETRIE: Etwas größer als Fenster für sauberen Durchbruch
    opening_profile_origin = model.createIfcCartesianPoint((0.0, 0.0))
    opening_profile_placement = model.createIfcAxis2Placement2D(opening_profile_origin)
    
    # Öffnung ist 2cm breiter als Fenster und durchstößt die ganze Wand
    opening_width = to_m(window_params["width"]) + 0.02    # +2cm breiter
    opening_depth = 0.25                                   # 25cm tief (> 20cm Wanddicke)
    
    opening_profile = model.createIfcRectangleProfileDef(
        "AREA", "Öffnungsprofil", opening_profile_placement,
        opening_width, opening_depth
    )
    
    # 3. ÖFFNUNGS-EXTRUSION: Etwas höher als Fenster
    opening_solid = model.createIfcExtrudedAreaSolid(
        opening_profile,
        opening_profile_placement,
        model.createIfcDirection((0.0, 0.0, 1.0)),
        to_m(window_params["height"]) + 0.01  # +1cm höher
    )
    
    # 4. ÖFFNUNGS-REPRÄSENTATION
    opening_shape = model.createIfcShapeRepresentation(
        body_context, "Body", "SweptSolid", [opening_solid]
    )
    opening_definition = model.createIfcProductDefinitionShape(None, None, [opening_shape])
    
    # 5. ÖFFNUNGSELEMENT: IfcOpeningElement erstellen
    opening = model.createIfcOpeningElement(
        ifcopenshell.guid.new(),    # Eindeutige ID
        owner_history,              # Änderungshistorie
        "Fensteröffnung",          # Name
        "Durchbruch für Hauptfenster", # Beschreibung
        None,                       # Objekttyp
        opening_placement,          # Platzierung
        opening_definition,         # Geometrie
        None,                       # Tag
        "OPENING"                   # Vordefinierter Typ
    )
    
    # 6. VOID-BEZIEHUNG: Öffnung "schneidet" die Wand
    model.createIfcRelVoidsElement(
        ifcopenshell.guid.new(),    # Eindeutige ID
        owner_history,              # Änderungshistorie
        "Wanddurchbruch",          # Name
        "Öffnung durchbricht Wand", # Beschreibung
        wall,                       # Voided Element (Wand)
        opening                     # Voiding Element (Öffnung)
    )
    print("✅ Wandöffnung erstellt und mit Wand verknüpft")
    
    # 7. FILL-BEZIEHUNG: Fenster "füllt" die Öffnung
    model.createIfcRelFillsElement(
        ifcopenshell.guid.new(),    # Eindeutige ID
        owner_history,              # Änderungshistorie
        "Fenster füllt Öffnung",   # Name
        "Fenster schließt den Durchbruch", # Beschreibung
        opening,                    # Relating Opening (Öffnung)
        window                      # Related Element (Fenster)
    )
    print("✅ Fenster mit Öffnung verknüpft")
    
    # 8. FENSTER-STYLING: Farbe und Transparenz
    print("🎨 Füge Fensterstil hinzu...")
    
    # Hellblaue Farbe definieren
    window_color = model.createIfcColourRgb(None, 0.7, 0.9, 1.0)
    
    # Oberflächenstil mit Transparenz
    surface_style = model.createIfcSurfaceStyleRendering(
        window_color,    # Oberflächenfarbe
        0.3,            # Transparenz (30%)
        window_color,    # Diffuse Farbe
        None, None, None, None, None,  # Weitere optionale Eigenschaften
        "FLAT"          # Reflexionsmethode
    )
    
    # Stil-Definition
    surface_styles = model.createIfcSurfaceStyle("Fensterstil", "BOTH", [surface_style])
    
    # Stil dem Fenster zuweisen (über die Geometrie)
    window_solid = None
    for shape_rep in window.Representation.Representations:
        if shape_rep.Items:
            window_solid = shape_rep.Items[0]
            break
    
    if window_solid:
        styled_item = model.createIfcStyledItem(window_solid, [surface_styles], "Fensterstil")
        print("✅ Fensterstil angewendet (hellblau, 30% transparent)")
    
    return opening

# Wandöffnung und Styling erstellen
opening = create_wall_opening_and_styling(model, owner_history, body_context, wall, window, window_params, to_m, world_placement)


## Schritt 5: Property Sets hinzufügen

### Was sind Property Sets?
Property Sets (Pset) erweitern IFC-Elemente um zusätzliche Eigenschaften und Metadaten:

**Standard Property Sets:**
- `Pset_WallCommon`: Standardeigenschaften für Wände (z.B. IsExternal, LoadBearing)
- `Pset_WindowCommon`: Standardeigenschaften für Fenster (z.B. IsExternal, Reference)

**Custom Property Sets:**
- `CPset_Custom`: Benutzerdefinierte Eigenschaften (z.B. Graffitischutz)
- `CPSet_ReUse`: Nachhaltigkeitseigenschaften (z.B. Wiederverwendbarkeit)

### Warum Property Sets verwenden?
- **Erweiterte Informationen**: Über Geometrie hinausgehende Daten
- **BIM-Integration**: Bessere Kompatibilität mit BIM-Software
- **Facility Management**: Wichtige Daten für Betrieb und Wartung
- **Nachhaltigkeit**: Informationen für Lebenszyklusanalysen


In [ ]:
def add_property_sets(model, owner_history, wall, window):
    """Fügt Standard- und benutzerdefinierte Property Sets hinzu"""
    
    print("📋 Füge Property Sets hinzu...")
    
    # 1. PSET_WALLCOMMON: Standard-Eigenschaften für Wände
    try:
        # Property Set für Wand erstellen
        wall_pset = model.createIfcPropertySet(
            ifcopenshell.guid.new(),     # Eindeutige ID
            owner_history,               # Änderungshistorie
            "Pset_WallCommon",          # Name (IFC-Standard)
            "Gemeinsame Eigenschaften für Wände", # Beschreibung
            []                          # Properties (werden unten hinzugefügt)
        )
        
        # Eigenschaft: IsExternal (Außenwand?)
        is_external_prop = model.createIfcPropertySingleValue(
            "IsExternal",                           # Name der Eigenschaft
            "Gibt an, ob die Wand eine Außenwand ist", # Beschreibung
            model.createIfcBoolean(True),           # Wert: True (Außenwand)
            None                                    # Einheit (nicht nötig für Boolean)
        )
        
        # Eigenschaft: LoadBearing (Tragend?)
        load_bearing_prop = model.createIfcPropertySingleValue(
            "LoadBearing",                          # Name
            "Gibt an, ob die Wand tragend ist",    # Beschreibung  
            model.createIfcBoolean(False),         # Wert: False (nicht tragend)
            None                                    # Einheit
        )
        
        # Properties dem Property Set hinzufügen
        wall_pset.HasProperties = [is_external_prop, load_bearing_prop]
        
        # Property Set mit Wand verknüpfen
        model.createIfcRelDefinesByProperties(
            ifcopenshell.guid.new(),     # Eindeutige ID
            owner_history,               # Änderungshistorie
            "Wand-Eigenschaften",       # Name der Beziehung
            "Verknüpfung von Pset_WallCommon mit Wand", # Beschreibung
            [wall],                      # Betroffene Objekte (Liste von Wänden)
            wall_pset                    # Definierendes Property Set
        )
        
        print("✅ Pset_WallCommon hinzugefügt (IsExternal=True, LoadBearing=False)")
        
    except Exception as e:
        print(f"⚠️  Warnung: Pset_WallCommon konnte nicht hinzugefügt werden: {e}")
    
    # 2. PSET_WINDOWCOMMON: Standard-Eigenschaften für Fenster
    try:
        # Property Set für Fenster erstellen
        window_pset = model.createIfcPropertySet(
            ifcopenshell.guid.new(),
            owner_history,
            "Pset_WindowCommon",
            "Gemeinsame Eigenschaften für Fenster",
            []
        )
        
        # Eigenschaft: IsExternal (Außenfenster?)
        is_external_prop = model.createIfcPropertySingleValue(
            "IsExternal",
            "Gibt an, ob das Fenster nach außen führt",
            model.createIfcBoolean(True),           # True = Außenfenster
            None
        )
        
        # Eigenschaft: Reference (Referenz-Bezeichnung)
        reference_prop = model.createIfcPropertySingleValue(
            "Reference",
            "Referenz-Bezeichnung des Fensters",
            model.createIfcLabel("FE-001"),         # Fenster-ID
            None
        )
        
        # Properties hinzufügen und verknüpfen
        window_pset.HasProperties = [is_external_prop, reference_prop]
        
        model.createIfcRelDefinesByProperties(
            ifcopenshell.guid.new(), owner_history,
            "Fenster-Eigenschaften", 
            "Verknüpfung von Pset_WindowCommon mit Fenster",
            [window], window_pset
        )
        
        print("✅ Pset_WindowCommon hinzugefügt (IsExternal=True, Reference=FE-001)")
        
    except Exception as e:
        print(f"⚠️  Warnung: Pset_WindowCommon konnte nicht hinzugefügt werden: {e}")
    
    # 3. CPSET_CUSTOM: Benutzerdefinierte Eigenschaften für Wand
    try:
        # Custom Property Set für Wand
        custom_pset = model.createIfcPropertySet(
            ifcopenshell.guid.new(),
            owner_history,
            "CPset_Custom",
            "Benutzerdefinierte Eigenschaften für Wände",
            []
        )
        
        # Eigenschaft: Graffitischutz
        graffiti_prop = model.createIfcPropertySingleValue(
            "Graffitischutz",
            "Gibt an, ob die Wand Graffitischutz hat",
            model.createIfcBoolean(True),           # True = Graffitischutz vorhanden
            None
        )
        
        custom_pset.HasProperties = [graffiti_prop]
        
        model.createIfcRelDefinesByProperties(
            ifcopenshell.guid.new(), owner_history,
            "Wand Custom-Eigenschaften",
            "Benutzerdefinierte Eigenschaften der Wand",
            [wall], custom_pset
        )
        
        print("✅ CPset_Custom hinzugefügt (Graffitischutz=True)")
        
    except Exception as e:
        print(f"⚠️  Warnung: CPset_Custom konnte nicht hinzugefügt werden: {e}")
    
    # 4. CPSET_REUSE: Nachhaltigkeitseigenschaften für Fenster
    try:
        # Nachhaltigkeits-Property Set für Fenster
        reuse_pset = model.createIfcPropertySet(
            ifcopenshell.guid.new(),
            owner_history,
            "CPSet_ReUse",
            "Wiederverwendungs-Eigenschaften für Fenster",
            []
        )
        
        # Eigenschaft: Reusable (Wiederverwendbar?)
        reusable_prop = model.createIfcPropertySingleValue(
            "Reusable",
            "Gibt an, ob das Element wiederverwendbar ist",
            model.createIfcBoolean(False),          # False = nicht wiederverwendbar
            None
        )
        
        reuse_pset.HasProperties = [reusable_prop]
        
        model.createIfcRelDefinesByProperties(
            ifcopenshell.guid.new(), owner_history,
            "Fenster Nachhaltigkeits-Eigenschaften",
            "Wiederverwendungseigenschaften des Fensters",
            [window], reuse_pset
        )
        
        print("✅ CPSet_ReUse hinzugefügt (Reusable=False)")
        
    except Exception as e:
        print(f"⚠️  Warnung: CPSet_ReUse konnte nicht hinzugefügt werden: {e}")

# Property Sets hinzufügen
add_property_sets(model, owner_history, wall, window)


## Schritt 6: Modell speichern und validieren

### Was passiert beim Speichern?
1. **IFC-Datei schreiben**: Alle Daten werden in das standardisierte IFC-Format konvertiert
2. **Validierung**: Überprüfung auf IFC4-Konformität und Vollständigkeit
3. **Download**: Datei wird für lokale Nutzung bereitgestellt

### Validierungsprüfungen:
- **Elementanzahl**: Kontrolle aller erstellten Objekte
- **Beziehungen**: Überprüfung der Verknüpfungen
- **Property Sets**: Validierung der Eigenschaften
- **Geometrie**: Kontrolle der 3D-Repräsentationen


In [ ]:
def save_and_validate_model(model):
    """Speichert das Modell und führt eine Validierung durch"""
    
    print("\n💾 Speichere IFC-Modell...")
    
    # 1. DATEINAME MIT ZEITSTEMPEL GENERIEREN
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"BFH_Vollmodell_{current_time}.ifc"
    
    # 2. IFC-DATEI SCHREIBEN
    model.write(filename)
    print(f"✅ IFC-Datei erfolgreich gespeichert: {filename}")
    
    # 3. VALIDIERUNG: Modell erneut öffnen und prüfen
    print("\n🔍 Validiere IFC-Modell...")
    test_model = ifcopenshell.open(filename)
    
    # Alle wichtigen Elemente zählen
    walls = test_model.by_type("IfcWall")
    windows = test_model.by_type("IfcWindow") 
    window_types = test_model.by_type("IfcWindowType")
    lining_props = test_model.by_type("IfcWindowLiningProperties")
    panel_props = test_model.by_type("IfcWindowPanelProperties")
    openings = test_model.by_type("IfcOpeningElement")
    void_rels = test_model.by_type("IfcRelVoidsElement")
    fill_rels = test_model.by_type("IfcRelFillsElement")
    type_rels = test_model.by_type("IfcRelDefinesByType")
    prop_rels = test_model.by_type("IfcRelDefinesByProperties")
    property_sets = test_model.by_type("IfcPropertySet")
    
    # 4. VALIDIERUNGSBERICHT
    print("📊 Modell-Statistiken:")
    print(f"   🧱 Wände: {len(walls)}")
    print(f"   🪟 Fenster: {len(windows)}")
    print(f"   🏷️  Fenstertypen: {len(window_types)}")
    print(f"   🖼️  Rahmeneigenschaften: {len(lining_props)}")
    print(f"   🔲 Panel-Eigenschaften: {len(panel_props)}")
    print(f"   ⚪ Öffnungen: {len(openings)}")
    print(f"   ✂️  Void-Beziehungen: {len(void_rels)}")
    print(f"   🔗 Fill-Beziehungen: {len(fill_rels)}")
    print(f"   🏗️  Typ-Beziehungen: {len(type_rels)}")
    print(f"   📋 Property-Beziehungen: {len(prop_rels)}")
    print(f"   📦 Property Sets: {len(property_sets)}")
    
    # 5. DETAILLIERTE VALIDIERUNG
    print("\n📚 Detailvalidierung:")
    
    # Wand-Validierung
    if len(walls) > 0:
        wall = walls[0]
        print(f"✅ Wand gefunden: '{wall.Name}' (ID: {wall.GlobalId})")
        
        # Property Sets der Wand prüfen
        wall_psets = []
        for rel in prop_rels:
            if wall in rel.RelatedObjects:
                if hasattr(rel.RelatingPropertyDefinition, 'Name'):
                    pset_name = rel.RelatingPropertyDefinition.Name
                    wall_psets.append(pset_name)
                    
                    # Spezifische Eigenschaften prüfen
                    if pset_name == "Pset_WallCommon":
                        pset = rel.RelatingPropertyDefinition
                        for prop in pset.HasProperties:
                            if prop.Name == "IsExternal":
                                print(f"   ✅ Wand IsExternal = {prop.NominalValue.wrappedValue}")
                            elif prop.Name == "LoadBearing":
                                print(f"   ✅ Wand LoadBearing = {prop.NominalValue.wrappedValue}")
                    elif pset_name == "CPset_Custom":
                        pset = rel.RelatingPropertyDefinition
                        for prop in pset.HasProperties:
                            if prop.Name == "Graffitischutz":
                                print(f"   ✅ Wand Graffitischutz = {prop.NominalValue.wrappedValue}")
        
        print(f"   📋 Wand Property Sets: {wall_psets}")
    else:
        print("❌ Keine Wand gefunden")
    
    # Fenster-Validierung
    if len(windows) > 0:
        window = windows[0]
        print(f"✅ Fenster gefunden: '{window.Name}' (ID: {window.GlobalId})")
        
        # Abmessungen prüfen
        if hasattr(window, 'OverallHeight') and window.OverallHeight:
            print(f"   ✅ Fensterhöhe: {window.OverallHeight}m")
        if hasattr(window, 'OverallWidth') and window.OverallWidth:
            print(f"   ✅ Fensterbreite: {window.OverallWidth}m")
        
        # Property Sets des Fensters prüfen
        window_psets = []
        for rel in prop_rels:
            if window in rel.RelatedObjects:
                if hasattr(rel.RelatingPropertyDefinition, 'Name'):
                    pset_name = rel.RelatingPropertyDefinition.Name
                    window_psets.append(pset_name)
                    
                    # Spezifische Eigenschaften prüfen
                    if pset_name == "Pset_WindowCommon":
                        pset = rel.RelatingPropertyDefinition
                        for prop in pset.HasProperties:
                            if prop.Name == "IsExternal":
                                print(f"   ✅ Fenster IsExternal = {prop.NominalValue.wrappedValue}")
                            elif prop.Name == "Reference":
                                print(f"   ✅ Fenster Reference = {prop.NominalValue.wrappedValue}")
                    elif pset_name == "CPSet_ReUse":
                        pset = rel.RelatingPropertyDefinition
                        for prop in pset.HasProperties:
                            if prop.Name == "Reusable":
                                print(f"   ✅ Fenster Reusable = {prop.NominalValue.wrappedValue}")
        
        print(f"   📋 Fenster Property Sets: {window_psets}")
    else:
        print("❌ Kein Fenster gefunden")
    
    # Beziehungs-Validierung
    if len(void_rels) > 0 and len(fill_rels) > 0:
        print("✅ Wandöffnung korrekt implementiert (Void + Fill)")
    else:
        print("❌ Wandöffnung unvollständig")
        
    if len(type_rels) >= 2:  # Wand + Fenster
        print("✅ Typ-Beziehungen korrekt definiert")
    else:
        print("❌ Typ-Beziehungen unvollständig")
    
    # 6. ERFOLGSMELDUNG
    print("\n🎉 Validierung abgeschlossen!")
    print("📁 Das Modell kann nun in BIM-Software (Revit, ArchiCAD, etc.) geöffnet werden")
    
    # 7. DATEI GESPEICHERT (lokale Ausführung)
    print(f"📁 Datei gespeichert: {filename}")
    # from google.colab import files
    # files.download(filename)
    
    return filename

# Modell speichern und validieren
final_filename = save_and_validate_model(model)


## 🎓 Zusammenfassung und Lernziele

### Was haben Sie in diesem Tutorial gelernt?

**✅ IFC-Grundlagen:**
- Vollständige IFC4-Modellstruktur von Grund auf erstellt
- Räumliche Hierarchie (Projekt → Standort → Gebäude → Geschoss) implementiert
- Geometrische Kontexte und Einheitensysteme definiert

**✅ Bauteil-Modellierung:**
- Wände mit korrekter Geometrie und Eigenschaften erstellt
- Fenster mit Typ-Definitionen und Abmessungen modelliert
- Wandöffnungen für realistische Durchbrüche implementiert

**✅ IFC-Beziehungen:**
- Void-Beziehungen (Wand ↔ Öffnung) korrekt verknüpft
- Fill-Beziehungen (Öffnung ↔ Fenster) implementiert
- Typ-Beziehungen für bessere Klassifizierung verwendet

**✅ Property Sets:**
- Standard Psets (`Pset_WallCommon`, `Pset_WindowCommon`) hinzugefügt
- Custom Psets (`CPset_Custom`, `CPSet_ReUse`) für erweiterte Daten erstellt
- Eigenschaften korrekt mit Elementen verknüpft

**✅ BIM-Standards:**
- IFC4-Schema vollständig eingehalten
- Kompatibilität mit BIM-Software sichergestellt
- Validierung und Qualitätskontrolle implementiert

### 🚀 Nächste Schritte für Ihre BIM-Reise:

1. **Experimentieren Sie!** 
   - Ändern Sie Abmessungen, Positionen oder Materialeigenschaften
   - Fügen Sie weitere Wände, Fenster oder Türen hinzu

2. **Erkunden Sie die Datei:**
   - Öffnen Sie das erstellte IFC-Modell in Revit, ArchiCAD oder einem IFC-Viewer
   - Untersuchen Sie die Property Sets und Beziehungen

3. **Erweitern Sie Ihr Wissen:**
   - Studieren Sie die [IFC4-Dokumentation](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/)
   - Lernen Sie über weitere IFC-Elementtypen (Türen, Treppen, MEP-Systeme)
   - Erkunden Sie IDS (Information Delivery Specification) für Datenvalidierung

4. **Praktische Anwendung:**
   - Erstellen Sie komplexere Gebäudemodelle
   - Integrieren Sie Facility Management Daten
   - Automatisieren Sie Planungsprozesse mit Python

### 📚 Weiterführende Ressourcen:

- **buildingSMART International**: [Standards und Dokumentation](https://www.buildingsmart.org/)
- **IFCOpenShell Dokumentation**: [API-Referenz und Beispiele](http://ifcopenshell.org/)
- **BIM-Software Integration**: Revit, ArchiCAD, Tekla, Bentley
- **Open Source Tools**: BlenderBIM, FreeCAD IFC Workbench

**Herzlichen Glückwunsch!** 🎉 Sie haben erfolgreich ein vollständiges, IFC4-konformes BIM-Modell erstellt!


### Fensterplatzierung
Nachdem wir die Wand erstellt haben, oeffnen wir die zuvor erstellte Datei und fuegen ein Fenster hinzu. Danach speichern wir wieder als neue Datei.

In [ ]:
import ifcopenshell
from ifcopenshell.api import run

# IFC-Modell laden - verwende final_filename aus dem vorherigen Schritt
model = ifcopenshell.open(final_filename)
storey = model.by_type("IfcBuildingStorey")[0] if model.by_type("IfcBuildingStorey") else None
if not storey:
    raise SystemExit("Keine Gebaeudeetage im Modell gefunden.")

# Wand finden
walls = model.by_type("IfcWall")
if not walls:
    raise SystemExit("Keine Wand im Modell gefunden.")
wall = walls[0]

# Fensterparameter (mm) -> wir konvertieren unten nach m
# Wand ist 5000mm lang x 200mm dick x 3000mm hoch
window_params = {
    "name": "Neues Fenster",
    "breite": 1800,
    "tiefe": 150,      # dünneres Fenster (150mm) damit es nicht durchsteckt
    "hoehe": 1200,
    "offset_x": 1600,  # Mitte der Wand: 2500mm - 900mm (halbe Fensterbreite) = 1600mm vom linken Rand
    "offset_y": 25,    # Fenster in Wanddicke zentrieren: (200mm-150mm)/2 = 25mm Versatz
    "offset_z": 1200   # höhere Platzierung (1200mm vom Boden, realistischere Fensterhöhe)
}

to_m = lambda v_mm: float(v_mm) / 1000.0

# Platzierung relativ zur Wand
wall_placement = wall.ObjectPlacement
wall_location = wall_placement.RelativePlacement.Location
placement_point = model.createIfcCartesianPoint((
    wall_location.Coordinates[0] + to_m(window_params["offset_x"]),
    wall_location.Coordinates[1] + to_m(window_params["offset_y"]),
    wall_location.Coordinates[2] + to_m(window_params["offset_z"]) ))
axis2placement = model.createIfcAxis2Placement3D(placement_point)
window_placement = model.createIfcLocalPlacement(wall_placement, axis2placement)

# Fenstergeometrie erstellen (rein visuell)
# Create proper 2D axis placement for the profile
profile_point = model.createIfcCartesianPoint((0.0, 0.0))
profile_axis = model.createIfcAxis2Placement2D(profile_point)
rect_profile = model.createIfcRectangleProfileDef("AREA", "BaseProfile", profile_axis, to_m(window_params["breite"]), to_m(window_params["tiefe"]))
extrude_dir = model.createIfcDirection((0.0, 0.0, 1.0))
extruded_area = model.createIfcExtrudedAreaSolid(rect_profile, profile_axis, extrude_dir, to_m(window_params["hoehe"]))
rep = model.createIfcShapeRepresentation(model.by_type("IfcGeometricRepresentationContext")[0], "Body", "SweptSolid", [extruded_area])
product_def_shape = model.createIfcProductDefinitionShape(None, None, [rep])

# Fensterentitaet
new_window = model.createIfcWindow(ifcopenshell.guid.new(), None, window_params["name"], None, None, window_placement, product_def_shape, None)
model.createIfcRelContainedInSpatialStructure(ifcopenshell.guid.new(), None, "Contains", None, [new_window], storey)

# Farbe und Transparenz vergeben (semi-transparent blue)
# Create color for window (semi-transparent blue)
window_color = model.createIfcColourRgb(None, 0.7, 0.9, 1.0)  # Light blue

# Create surface style rendering with transparency
surface_style = model.createIfcSurfaceStyleRendering(
    window_color,        # SurfaceColour
    0.3,                # Transparency (30% transparent)
    window_color,       # DiffuseColour
    None,               # TransmissionColour
    None,               # DiffuseTransmissionColour
    None,               # ReflectionColour
    None,               # SpecularColour
    None,               # SpecularHighlight
    "FLAT"             # ReflectanceMethod (enumeration)
)

# Create surface style
surface_styles = model.createIfcSurfaceStyle("WindowStyle", "BOTH", [surface_style])

# Create styled item to apply style to window geometry
styled_item = model.createIfcStyledItem(extruded_area, [surface_styles], "WindowStyle")

# Wandöffnung erstellen (für korrekte Wanddurchdringung)
# Create opening placement (same as window placement)
opening_point = model.createIfcCartesianPoint((
    wall_location.Coordinates[0] + to_m(window_params["offset_x"]),
    wall_location.Coordinates[1] + to_m(window_params["offset_y"]),
    wall_location.Coordinates[2] + to_m(window_params["offset_z"])
))
opening_placement = model.createIfcLocalPlacement(wall_placement,
    model.createIfcAxis2Placement3D(opening_point)
)

# Create opening profile (slightly larger than window for proper cutout)
opening_profile_origin = model.createIfcCartesianPoint((0.0, 0.0))
opening_profile_placement = model.createIfcAxis2Placement2D(opening_profile_origin)

# Make opening larger than window AND wall thickness for proper cutout
# Wall is 200mm (0.2m) thick, so opening needs to be thicker than that
opening_width = to_m(window_params["breite"]) + 0.02  # +2cm wider than window
opening_depth = 0.25  # 250mm thick opening (larger than 200mm wall)
opening_profile = model.createIfcRectangleProfileDef(
    "AREA",
    "OpeningProfile",
    opening_profile_placement,
    opening_width,
    opening_depth
)

# Create opening solid
opening_solid = model.createIfcExtrudedAreaSolid(
    opening_profile,
    opening_profile_placement,
    model.createIfcDirection((0.0, 0.0, 1.0)),
    to_m(window_params["hoehe"]) + 0.01  # Slightly taller
)

# Create opening shape representation
opening_shape = model.createIfcShapeRepresentation(
    model.by_type("IfcGeometricRepresentationContext")[0],
    "Body",
    "SweptSolid",
    [opening_solid]
)
opening_definition = model.createIfcProductDefinitionShape(None, None, [opening_shape])

# Create opening element
opening = model.createIfcOpeningElement(
    ifcopenshell.guid.new(),
    None,
    "Fensteroeffnung",
    None, None,
    opening_placement,
    opening_definition,
    None, None
)

# Create void relationship between wall and opening
model.createIfcRelVoidsElement(
    ifcopenshell.guid.new(),
    None,
    "Wand Oeffnung",
    None,
    wall,
    opening
)

# Speichern
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"Wand_mit_Fenster_{current_time}.ifc"
model.write(filename)
print("Updated model saved to:", filename)
# from google.colab import files
# files.download(filename)

### Fensteroeffnung in die Wand schneiden (Schritt 3)
In diesem Schritt wird eine `IfcOpeningElement` erstellt und ueber `IfcRelVoidsElement` mit der Wand verknuepft. Dadurch entsteht eine echte Oeffnung in der Wandgeometrie. Wir nutzen dieselben Parameter und Offsets wie beim Fenster.

In [ ]:
import ifcopenshell
import datetime

# Vorherige Ausgabedatei oeffnen - verwende final_filename aus dem vorherigen Schritt
model = ifcopenshell.open(final_filename)

# Wand holen
walls = model.by_type("IfcWall")
if not walls:
    raise SystemExit("Keine Wand im Modell gefunden.")
wall = walls[0]

# Sicherstellen, dass window_params existiert (aus Schritt 2)
try:
    window_params
except NameError:
    # Fallback: Standardwerte falls das Notebook direkt in Schritt 3 gestartet wird
    window_params = {
        "breite": 1800,
        "tiefe": 250,
        "hoehe": 1200,
        "offset_x": 1500,
        "offset_y": 50,
        "offset_z": 500
    }

to_m = lambda v_mm: float(v_mm) / 1000.0

# Platzierung fuer die Oeffnung relativ zur Wand wie beim Fenster
wall_placement = wall.ObjectPlacement
wall_location = wall_placement.RelativePlacement.Location
opening_point = model.createIfcCartesianPoint((
    wall_location.Coordinates[0] + to_m(window_params["offset_x"]),
    wall_location.Coordinates[1] + to_m(window_params["offset_y"]),
    wall_location.Coordinates[2] + to_m(window_params["offset_z"]) ))
opening_axis = model.createIfcAxis2Placement3D(opening_point)

# Profil und Extrusion fuer die Oeffnung (Breite x Tiefe, Extrusion in Z = Hoehe)
# Tiefe darf die Wand staerke ueberschreiten, damit sicher voll ausgeschnitten wird
opening_depth_m = to_m(window_params["tiefe"])  # z.B. 0.25 m fuer 0.2 m Wand
rect_open = model.createIfcRectangleProfileDef("AREA", "OpeningProfile", opening_axis, to_m(window_params["breite"]), opening_depth_m)
extrude_dir = model.createIfcDirection((0.0, 0.0, 1.0))
opening_solid = model.createIfcExtrudedAreaSolid(rect_open, opening_axis, extrude_dir, to_m(window_params["hoehe"]))

# Repr. fuer die Oeffnung anlegen
geom_context = model.by_type("IfcGeometricRepresentationContext")[0]
rep_open = model.createIfcShapeRepresentation(geom_context, "Body", "SweptSolid", [opening_solid])
pds_open = model.createIfcProductDefinitionShape(None, None, [rep_open])

# IfcOpeningElement erzeugen und mit Wand verknuepfen
opening = model.createIfcOpeningElement(ifcopenshell.guid.new(), None, "Fensteroeffnung", None, None, wall_placement, pds_open, None)
model.createIfcRelVoidsElement(ifcopenshell.guid.new(), None, "Voids", None, wall, opening)

# Datei erneut speichern
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"Wand_mit_Fenster_Oeffnung_{current_time}.ifc"
model.write(filename)
print("Model mit echter Oeffnung gespeichert unter:", filename)

# from google.colab import files
# files.download(filename)

### Bereinigung des Arbeitsbereichs
Nachdem das Modell gespeichert und heruntergeladen wurde, ist es oft sinnvoll, den Arbeitsbereich aufzuraeumen, indem man nicht mehr benoetigte Dateien entfernt. Dies hilft, die Uebersichtlichkeit zu bewahren und Speicherplatz freizugeben.

In [ ]:
import os

for file_name in os.listdir():
    if file_name.endswith('.ifc'):
        os.remove(os.path.join(os.getcwd(), file_name))
        print(f"Removed {file_name}")
